# Job planet 채용 공고 scrapping 스크립트

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import pandas as pd

import time
from tqdm.notebook import tqdm

In [2]:
baseUrl = 'https://www.jobplanet.co.kr'
scroll_num = 30 # 9 + 9 * scroll_num of hrefs in maximum
major_classification = '데이터'
minor_classification = '데이터 전체'

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [3]:
# Function for scrolling

def scroll(driver, scroll_limit):
    '''
    Each scroll gets 9 additional references and it takes 1 second of timeout.
    '''
    n = 1
    prev_height = driver.execute_script('return document.body.scrollHeight')
    while n <= scroll_limit:
        print('scrolling', n, '/', scroll_limit)
        n += 1
        driver.execute_script('window.scrollBy(0, 3000)')
        time.sleep(1)
        curr_height = driver.execute_script('return document.body.scrollHeight')
        if curr_height == prev_height:
            time.sleep(5)
            curr_height = driver.execute_script('return document.body.scrollHeight')
            if curr_height == prev_height:
                break
        prev_height = curr_height
    time.sleep(5)

In [4]:
# Making a list of target hrefs (job_descriptions)

with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
  driver.implicitly_wait(5)
  driver.get(baseUrl + '/job')

  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'jply_checkbox_item')))
  driver.find_element(By.XPATH, "//*[text()='직종']").click()
  driver.find_element(By.XPATH, f"//*[text()='{major_classification}']").click()
  driver.find_element(By.XPATH, f"//*[text()='{minor_classification}']").click()
  driver.find_element(By.XPATH, "//*[text()='적용']").click()

  time.sleep(5)

  scroll(driver, scroll_num) # get (1 + 3) * 9 = 36 job descriptions

  print('collecting hrefs')
  job_descriptions = [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, 'a') if a.get_attribute('href') and baseUrl + '/job/search' in a.get_attribute('href')]

print(len(job_descriptions), job_descriptions)

scrolling 1 / 30
scrolling 2 / 30
scrolling 3 / 30
scrolling 4 / 30
scrolling 5 / 30
scrolling 6 / 30
scrolling 7 / 30
scrolling 8 / 30
scrolling 9 / 30
scrolling 10 / 30
scrolling 11 / 30
scrolling 12 / 30
collecting hrefs
108 ['https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250649', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250473', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250480', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250323', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250321', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250417', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250400', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250710', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250346', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250239', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250241', 'https://www.jobplanet.

In [5]:
def find_by_text(html, text, data_obj, col_name, parent=None, h3=None):
    finding = html.find(name=h3, text=text)

    if finding and parent:
        finding = finding.find_parent('dt')

    if finding:
        data_obj[col_name].append(finding.find_next_sibling().text)
    else:
        data_obj[col_name].append('')

In [6]:
# Scrapping recruitment data from all hrefs collected

extracted_data = {
    'href': [],
    'title': [],
    'company_name': [],
    'job_location': [],
    'job_classification': [],
    'due_date': [],
    'work_experience': [],
    'skills': [],
    'company_intro': [],
    'main_task': [],
    'qualification': [],
    'preference': [],
    'procedure': [],
    'benefits': [],
    'job_location_detail': [],
    'detail_description': [],
}


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.implicitly_wait(5)
    for i in tqdm(range(len(job_descriptions))):
      driver.get(job_descriptions[i])
      WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'job_apply_section')))

      soup = BeautifulSoup(driver.page_source, 'html.parser')
      apply = soup.find('div', 'job_apply_section')
      detail = soup.find('section', 'recruitment-detail')
      summary = detail.find('dl', 'recruitment-summary__dl')

      extracted_data['href'].append(job_descriptions[i])
      extracted_data['title'].append(apply.find('h1', 'ttl').text)
      extracted_data['company_name'].append(apply.find('span', 'company_name').text)
      extracted_data['job_location'].append(apply.find('span', 'job_location').text)
      find_by_text(summary, '직무', extracted_data, 'job_classification', parent=True)
      find_by_text(summary, '마감일', extracted_data, 'due_date', parent=True)
      find_by_text(summary, '경력', extracted_data, 'work_experience', parent=True)
      find_by_text(summary, '스킬', extracted_data, 'skills', parent=True)
      find_by_text(detail, '기업 소개', extracted_data, 'company_intro', h3=True)
      find_by_text(detail, '주요 업무', extracted_data, 'main_task', h3=True)
      find_by_text(detail, '자격 요건', extracted_data, 'qualification', h3=True)
      find_by_text(detail, '우대사항', extracted_data, 'preference', h3=True)
      find_by_text(detail, '채용 절차', extracted_data, 'procedure', h3=True)
      find_by_text(detail, '복리후생', extracted_data, 'benefits', h3=True)
      find_by_text(detail, '회사위치', extracted_data, 'job_location_detail', h3=True)
      extracted_data['detail_description'].append(detail)


  0%|          | 0/108 [00:00<?, ?it/s]

In [7]:
extracted_df = pd.DataFrame(extracted_data)
extracted_df.head()

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,procedure,benefits,job_location_detail,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,DB 관리 (카드IS 3팀),롯데정보통신(주),서울,데이터 엔지니어,2023.07.03 D-7,3년 이상,"SQL, MYSQL, DB, 데이터베이스","오늘을 새롭게, 내일을 이롭게. 롯데\n\n롯데와 함께 비전을 공유할 수 있는 유능...",ㆍSQL 튜닝\nㆍDatabase Admin\nㆍDB Performance 튜닝,ㆍ경력 3년 이상\nㆍOracle Database 운영/백업/모니터링/장애처리 경험...,ㆍSQL 튜닝\nㆍOGG 솔루션 운영 경험\nㆍ금융권 대용량 DB시스템 운영 경험\...,지원서접수 > 서류전형 > 직무테스트/인성진단(온라인) > 면접전형 > 평판조회 >...,ㆍ복지포인트 및 경조지원\nㆍ건강검진\nㆍ학자금/교육지원 등 다양한 복리후생 지원\...,서울 금천구 가산동 533-2 롯데센터 15층,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
1,https://www.jobplanet.co.kr/job/search?posting...,DB & 데이터 Engineer,(주)비트나인,서울,데이터 엔지니어,2023.06.26 오늘 마감,경력무관,"데이터, DB, rdbms, 데이터엔지니어",비트나인에서 엔터프라이즈 사업부 영업대표 담당자를 모집합니다.\n\n2013년 설립...,"· 당사 DBMS제품 설치구축, 트러블슈팅, 튜닝 기술지원\n· RDBMS(Agen...",· 학력 : 4년재 대학 졸업 이상\n· 경력 : 해당 DBMS 분야의 엔지니어 또...,· DBMS에 대한 애정과 역량을 발휘하고 싶은 경력자\n· DBMS QA(DBMS...,,"• 복지포인트를 지원해줘요(가족식비,여가비,취미활동비 등 연240만원 수준)\n• ...","서울 강남구 테헤란로 516, 4층","[[[요약], [<dt class=""recruitment-summary__dt""><..."
2,https://www.jobplanet.co.kr/job/search?posting...,Data Analyst (데이터 분석가),(주)모요,서울,데이터 분석가,2023.07.21 상시채용,3년 이상,"데이터분석, python, aws, SQL, 데이터 분석, 파이썬, gcp, Tab...",# 모두에게 통신을 쉽고 정직하게\n\n모요는 어려운 통신 용어와 절차를 쉽게 안내...,# 데이터 분석 환경은 이렇게 되어있어요\n- 모요는 자체적인 Data Lake를 ...,# 모요와 함께해요\n\n< 이런 분을 찾고있어요 >\n- 3년 이상의 프로덕트 데...,< 이런 분이면 더 좋아요 >\n- 클라우드 환경에서 데이터 분석을 해본 분(AWS...,# 합류 여정 \n지원서 접수 → 서류 검토 → 쿼리 테스트(SQL) 및 실무 인터...,＜물리적 환경 지원＞\n• 역삼역 3번출구에서 도보 2분 거리의 오피스에 입주해있어...,"서울 강남구 테헤란로 217 역삼동 오렌지플래닛, 2층","[[[요약], [<dt class=""recruitment-summary__dt""><..."
3,https://www.jobplanet.co.kr/job/search?posting...,임상 Data management 및 Web Scripting (EDC 관리 - D...,(주)엠투커뮤니티,서울,데이터 분석가,2023.06.30 D-4,2년 이상,"데이터분석, SQL, R, 임상, DM",M2community는 IT기술과 메디컬 콘텐츠의 컨버전스를 통해 더 나은 솔루션 ...,임상 DM (Data Management)\n ㆍEDC 개발 지원 (DB 설계...,ㆍSQL scripting에 익숙한 자\nㆍData Handling 가능자\nㆍMS...,ㆍDM 경력 2년 이상 경력자 (DM 업무 능숙자)\nㆍEDC 개발 및 운영 경험자...,서류전형 > 면접전형 > 최종합격,"급여제도 \n퇴직연금, 4대보험, 휴일(특근)수당, 인센티브 제도\n\n휴가제도\n...","서울 성동구 성수이로 66, 807호","[[[요약], [<dt class=""recruitment-summary__dt""><..."
4,https://www.jobplanet.co.kr/job/search?posting...,임상연구 DM - PM Part 추가 모집 (CRC 경력 우대),(주)엠투커뮤니티,서울,데이터 분석가,2023.06.30 D-4,2년 이상,"pm, 데이터관리, CRA, 임상, CRC, DM",M2community는 IT기술과 메디컬 콘텐츠의 컨버전스를 통해 더 나은 솔루션 ...,임상 DM - PM Part\n ㆍCase report form 개발 및 운영...,ㆍCRC 및 간호 경험자 또는 의학 용어에 익숙한 자\nㆍ임상시험 Data Hand...,ㆍCRA 또는 PM 경력자 (Project Lead 경험자)\nㆍ제약회사 / CRO...,서류전형 > 면접전형 > 최종합격,"급여제도 \n퇴직연금, 4대보험, 휴일(특근)수당, 인센티브 제도\n\n휴가제도\n...","서울 성동구 성수이로 66, 807호","[[[요약], [<dt class=""recruitment-summary__dt""><..."


In [9]:
extracted_df.to_csv('./extracted.csv', index=False)